In [32]:
import pandas as pd
import urllib.parse
import os

In [33]:
!pip install wwo-hist

In [34]:
from wwo_hist import retrieve_hist_data

In [18]:
import random
from datetime import datetime, timedelta

# Function to generate random dates within a range of years
def generate_random_dates(start_year, end_year, num_dates):
    dates = []
    for _ in range(num_dates):
        year = random.randint(start_year, end_year)
        month = random.randint(1, 12)
        day = random.randint(1, 28)  # Assuming all months have 28 days for simplicity
        dates.append(datetime(year, month, day))
    return dates

# List of dates in the format 'YYYY-MM-DD'
existing_dates = df['date_time'].unique().tolist() 

# Convert existing dates to datetime objects for comparison
existing_dates = [datetime.strptime(date_str, '%Y-%m-%d') for date_str in existing_dates]




Unique random dates: ['2000-09-02', '2015-04-03', '2006-01-09', '2022-09-03', '2010-05-10', '2005-01-27', '2020-03-07', '2025-06-13', '2024-07-16', '2023-06-03']


In [22]:
# Define the range of years for random date generation
start_year = 2008
end_year = 2014

# Generate 10 random dates that are not in the existing list
num_dates_to_generate = 200
random_dates = generate_random_dates(start_year, end_year, num_dates_to_generate)

# Filter out dates that are already in the existing list
unique_random_dates = [date.strftime('%Y-%m-%d') for date in random_dates if date not in existing_dates]

In [21]:
df = pd.read_csv('combined_data.csv')

In [36]:
df['target'] = 1

In [37]:
df

,date_time,maxtempC,mintempC,totalSnow_cm,sunHour,uvIndex,moon_illumination,moonrise,moonset,sunrise,...,cloudcover,humidity,precipMM,pressure,tempC,visibility,winddirDegree,windspeedKmph,location,target
0,2008-07-10,25,16,0.0,8.0,4,47,01:52 PM,12:33 AM,06:22 AM,...,68,91,25.3,1014,25,8,169,7,San%20Francisco%20Bay%20Area,1
1,2008-07-10,25,16,0.0,8.0,4,47,01:52 PM,12:33 AM,06:22 AM,...,68,91,25.3,1014,25,8,169,7,San%20Francisco%20Bay%20Area,1
2,2014-06-30,30,22,0.0,10.0,6,16,08:49 AM,10:31 PM,05:34 AM,...,63,92,28.8,1006,30,7,198,16,Illinois,1
3,2014-07-01,26,18,0.0,15.0,6,23,09:46 AM,11:01 PM,05:34 AM,...,19,82,0.0,1007,26,10,265,17,Illinois,1
4,2014-07-02,18,14,0.0,5.0,4,30,10:42 AM,11:30 PM,05:35 AM,...,81,90,1.1,1015,18,7,297,13,Illinois,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
377,2008-11-11,27,20,0.0,10.0,5,93,04:20 PM,03:57 AM,05:44 AM,...,62,92,9.2,1010,27,8,223,4,Island%20of%20Puerto%20Rico,1
378,2008-11-07,22,10,0.0,11.0,5,62,01:55 PM,12:33 AM,06:45 AM,...,2,73,0.0,1013,22,10,220,9,Southern%20California,1
379,2008-09-22,26,25,0.0,4.0,6,39,No moonrise,01:24 PM,06:12 AM,...,85,88,94.3,1013,26,5,131,19,Island%20of%20Puerto%20Rico,1
380,2008-08-16,33,24,0.0,11.0,6,100,07:40 PM,06:40 AM,06:58 AM,...,43,78,16.8,1012,33,8,76,5,Island%20of%20Puerto%20Rico,1


In [30]:
import urllib.parse
import os
import pandas as pd

def process_weather_data(api_key, t):
    for index, row in t.iterrows():
        print(index)
        # Extract values from the row
        location = row['Location']
        encoded_location = urllib.parse.quote(location)
        print(encoded_location)
        start_date = row['Date']
        end_date = row['Date']  # Use a different date for the end date
        frequency = 24

        try:
            # Call retrieve_hist_data function to get historical weather data
            response = retrieve_hist_data(api_key=api_key,
                                           location_list=[encoded_location],
                                           start_date=start_date,
                                           end_date=end_date,
                                           frequency=frequency)

            if response is None:
                print(f"No data returned for location: {location}")
                continue  # Skip to the next iteration if no data is returned

            # Continue processing the data if it's successfully retrieved

            s = row['Geographic Areas'].replace(' ', '%20') + ".csv"
            df_excel = pd.read_csv(s)

            # Read existing data from combined_data.csv into another DataFrame
            combined_data_csv_path = 'combined_data.csv'
            if os.path.exists(combined_data_csv_path):
                df_combined_data = pd.read_csv(combined_data_csv_path)
            else:
                df_combined_data = pd.DataFrame()  # If combined_data.csv doesn't exist yet

            # Concatenate new data from Excel file with existing data from combined_data.csv
            combined_df = pd.concat([df_combined_data, df_excel], ignore_index=True)

            # Write the combined DataFrame back to combined_data.csv
            combined_df.to_csv(combined_data_csv_path, index=False)

        except Exception as e:
            print(f"Error processing data for location: {location}. Error: {e}")
            continue  # Skip to the next iteration if an error occurs


In [10]:
decoded_locations = [urllib.parse.unquote(location) for location in all_locations]

In [11]:
decoded_locations

['San Francisco Bay Area',
 'Illinois',
 'Alberta',
 'Texas',
 'North Carolina',
 'Michigan',
 'New York',
 'California',
 'Northern California',
 'Pennsylvania',
 'Ohio',
 'Southeastern Michigan',
 'Newark',
 'Tacoma',
 'Virginia',
 'Northern Illinois',
 'Georgia',
 'Southern California',
 'Northern and Central California',
 'Island of Puerto Rico']

In [17]:
import urllib.parse
import pandas as pd


# List of all locations and dates you want to retrieve data for
all_locations = df['location'].unique().tolist()  # List of all locations you're interested in
all_locations = [urllib.parse.unquote(location) for location in all_locations]
all_dates = unique_random_dates      # List of all dates you're interested in






286
0


In [ ]:
unique_random_dates

In [25]:
import numpy as np
# Assuming you have all_locations and all_dates defined

# Randomly select a location for each date
selected_locations = [np.random.choice(all_locations) for _ in range(len(all_dates))]

# Create a DataFrame with 200 rows (dates) and a single column for the selected locations
random_table = pd.DataFrame({'Date': all_dates, 'Location': selected_locations})



           Date                         Location
0    2008-07-10              Northern California
1    2014-06-30              Southern California
2    2014-07-01                         Virginia
3    2014-07-02            Island of Puerto Rico
4    2014-06-09                          Alberta
..          ...                              ...
281  2009-03-08                            Texas
282  2008-11-11                          Alberta
283  2008-11-07                         Michigan
284  2008-09-22  Northern and Central California
285  2008-08-16                         New York

[286 rows x 2 columns]


In [26]:
random_table

,Date,Location
0,2008-07-10,Northern California
1,2014-06-30,Southern California
2,2014-07-01,Virginia
3,2014-07-02,Island of Puerto Rico
4,2014-06-09,Alberta
...,...,...
281,2009-03-08,Texas
282,2008-11-11,Alberta
283,2008-11-07,Michigan
284,2008-09-22,Northern and Central California


In [39]:
api_key = '07197fcd9b464a3ca62233857241702'
# process_weather_data(api_key, random_table)

In [45]:
# Assuming t is your DataFrame
for index, row in random_table.iterrows():
    print(index)
    # Extract values from the row
    api_key = '07197fcd9b464a3ca62233857241702'
    location = row['Location']
    encoded_location = urllib.parse.quote(location)
    print(encoded_location)
    start_date = row['Date']
    end_date = row['Date']  # Use a different date for the end date
    frequency = 24

    try:
        # Call retrieve_hist_data function to get historical weather data
        response = retrieve_hist_data(api_key=api_key,
                                       location_list=[encoded_location],
                                       start_date=start_date,
                                       end_date=end_date,
                                       frequency=frequency)
        
        if response is None:
            print(f"No data returned for location: {location}")
            continue  # Skip to the next iteration if no data is returned

        # Continue processing the data if it's successfully retrieved
        
        s = row['Location'].replace(' ', '%20') + ".csv"
        df_excel = pd.read_csv(s)

        # Read existing data from combined_data.csv into another DataFrame
        combined_data_csv_path = 'combined_data.csv'
        if os.path.exists(combined_data_csv_path):
            df_combined_data = pd.read_csv(combined_data_csv_path)
        else:
            df_combined_data = pd.DataFrame()  # If combined_data.csv doesn't exist yet

        # Concatenate new data from Excel file with existing data from combined_data.csv
        combined_df = pd.concat([df_combined_data, df_excel], ignore_index=True)

        # Write the combined DataFrame back to combined_data.csv
        combined_df.to_csv(combined_data_csv_path, index=False)
        
    except Exception as e:
        print(f"Error processing data for location: {location}. Error: {e}")
        continue  # Skip to the next iteration if an error occurs

0
Northern%20California


Retrieving weather data for Northern%20California


Currently retrieving data for Northern%20California: from 2008-07-10 to 2008-07-10


C:\Users\Heliiix\anaconda3\Lib\site-packages\wwo_hist\__init__.py:69: FutureWarning: Argument `closed` is deprecated in favor of `inclusive`.
  list_mon_begin = pd.date_range(start_date, end_date, freq='MS', closed='right')
C:\Users\Heliiix\anaconda3\Lib\site-packages\wwo_hist\__init__.py:74: FutureWarning: Argument `closed` is deprecated in favor of `inclusive`.
  list_mon_end = pd.date_range(start_date, end_date, freq='M', closed='left')


Time elapsed (hh:mm:ss.ms) 0:00:00.566024


export Northern%20California completed!


1
Southern%20California


Retrieving weather data for Southern%20California


Currently retrieving data for Southern%20California: from 2014-06-30 to 2014-06-30


C:\Users\Heliiix\anaconda3\Lib\site-packages\wwo_hist\__init__.py:69: FutureWarning: Argument `closed` is deprecated in favor of `inclusive`.
  list_mon_begin = pd.date_range(start_date, end_date, freq='MS', closed='right')
C:\Users\Heliiix\anaconda3\Lib\site-packages\wwo_hist\__init__.py:74: FutureWarning: Argument `closed` is deprecated in favor of `inclusive`.
  list_mon_end = pd.date_range(start_date, end_date, freq='M', closed='left')


Time elapsed (hh:mm:ss.ms) 0:00:00.505905


export Southern%20California completed!


2
Virginia


Retrieving weather data for Virginia


Currently retrieving data for Virginia: from 2014-07-01 to 2014-07-01


...

Time elapsed (hh:mm:ss.ms) 0:00:00.234207


export Michigan completed!


284
Northern%20and%20Central%20California


Retrieving weather data for Northern%20and%20Central%20California


Currently retrieving data for Northern%20and%20Central%20California: from 2008-09-22 to 2008-09-22


C:\Users\Heliiix\anaconda3\Lib\site-packages\wwo_hist\__init__.py:69: FutureWarning: Argument `closed` is deprecated in favor of `inclusive`.
  list_mon_begin = pd.date_range(start_date, end_date, freq='MS', closed='right')
C:\Users\Heliiix\anaconda3\Lib\site-packages\wwo_hist\__init__.py:74: FutureWarning: Argument `closed` is deprecated in favor of `inclusive`.
  list_mon_end = pd.date_range(start_date, end_date, freq='M', closed='left')


Time elapsed (hh:mm:ss.ms) 0:00:00.321306


export Northern%20and%20Central%20California completed!


285
New%20York


Retrieving weather data for New%20York


Currently retrieving data for New%20York: from 2008-08-16 to 2008-08-16


C:\Users\Heliiix\anaconda3\Lib\site-packages\wwo_hist\__init__.py:69: FutureWarning: Argument `closed` is deprecated in favor of `inclusive`.
  list_mon_begin = pd.date_range(start_date, end_date, freq='MS', closed='right')
C:\Users\Heliiix\anaconda3\Lib\site-packages\wwo_hist\__init__.py:74: FutureWarning: Argument `closed` is deprecated in favor of `inclusive`.
  list_mon_end = pd.date_range(start_date, end_date, freq='M', closed='left')


Time elapsed (hh:mm:ss.ms) 0:00:00.261344


export New%20York completed!




In [46]:
dfNew = pd.read_csv('combined_data.csv')

In [48]:
dfNew['target'] = 0

In [49]:
dfNew

,date_time,maxtempC,mintempC,totalSnow_cm,sunHour,uvIndex,moon_illumination,moonrise,moonset,sunrise,...,cloudcover,humidity,precipMM,pressure,tempC,visibility,winddirDegree,windspeedKmph,location,target
0,2008-07-10,27,14,0.0,15.0,6,47,02:16 PM,12:21 AM,05:54 AM,...,20,68,0.0,1017,27,9,314,9,Northern%20California,0
1,2014-06-30,32,19,0.0,14.0,7,16,09:10 AM,10:36 PM,06:06 AM,...,25,68,0.0,1016,32,10,174,7,Southern%20California,0
2,2014-07-02,30,26,0.0,12.0,6,30,10:09 AM,10:41 PM,05:52 AM,...,36,77,0.5,1017,30,10,88,16,Island%20of%20Puerto%20Rico,0
3,2014-06-09,27,16,0.0,9.0,5,81,03:27 PM,02:33 AM,05:38 AM,...,65,83,13.7,1012,27,8,197,3,Alberta,0
4,2014-06-03,27,17,0.0,13.0,5,35,11:05 AM,12:04 AM,05:42 AM,...,46,82,0.3,1013,27,8,248,12,Pennsylvania,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
272,2009-03-08,24,19,0.0,12.0,6,81,04:08 PM,04:53 AM,06:37 AM,...,23,81,0.0,1017,24,10,164,16,Texas,0
273,2008-11-11,27,12,0.0,11.0,5,93,04:14 PM,04:22 AM,06:09 AM,...,22,79,0.0,1013,27,7,95,3,Alberta,0
274,2008-11-07,13,4,0.0,10.0,2,62,01:32 PM,No moonset,06:27 AM,...,33,65,0.0,1007,13,10,213,17,Michigan,0
275,2008-09-22,23,12,0.0,12.0,4,39,No moonrise,03:23 PM,07:03 AM,...,39,77,0.0,1027,23,7,58,9,Northern%20and%20Central%20California,0


In [51]:
df

,date_time,maxtempC,mintempC,totalSnow_cm,sunHour,uvIndex,moon_illumination,moonrise,moonset,sunrise,...,cloudcover,humidity,precipMM,pressure,tempC,visibility,winddirDegree,windspeedKmph,location,target
0,2008-07-10,25,16,0.0,8.0,4,47,01:52 PM,12:33 AM,06:22 AM,...,68,91,25.3,1014,25,8,169,7,San%20Francisco%20Bay%20Area,1
1,2008-07-10,25,16,0.0,8.0,4,47,01:52 PM,12:33 AM,06:22 AM,...,68,91,25.3,1014,25,8,169,7,San%20Francisco%20Bay%20Area,1
2,2014-06-30,30,22,0.0,10.0,6,16,08:49 AM,10:31 PM,05:34 AM,...,63,92,28.8,1006,30,7,198,16,Illinois,1
3,2014-07-01,26,18,0.0,15.0,6,23,09:46 AM,11:01 PM,05:34 AM,...,19,82,0.0,1007,26,10,265,17,Illinois,1
4,2014-07-02,18,14,0.0,5.0,4,30,10:42 AM,11:30 PM,05:35 AM,...,81,90,1.1,1015,18,7,297,13,Illinois,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
377,2008-11-11,27,20,0.0,10.0,5,93,04:20 PM,03:57 AM,05:44 AM,...,62,92,9.2,1010,27,8,223,4,Island%20of%20Puerto%20Rico,1
378,2008-11-07,22,10,0.0,11.0,5,62,01:55 PM,12:33 AM,06:45 AM,...,2,73,0.0,1013,22,10,220,9,Southern%20California,1
379,2008-09-22,26,25,0.0,4.0,6,39,No moonrise,01:24 PM,06:12 AM,...,85,88,94.3,1013,26,5,131,19,Island%20of%20Puerto%20Rico,1
380,2008-08-16,33,24,0.0,11.0,6,100,07:40 PM,06:40 AM,06:58 AM,...,43,78,16.8,1012,33,8,76,5,Island%20of%20Puerto%20Rico,1


In [53]:
concatenated_df = pd.concat([df, dfNew], ignore_index=True)

In [54]:
concatenated_df

,date_time,maxtempC,mintempC,totalSnow_cm,sunHour,uvIndex,moon_illumination,moonrise,moonset,sunrise,...,cloudcover,humidity,precipMM,pressure,tempC,visibility,winddirDegree,windspeedKmph,location,target
0,2008-07-10,25,16,0.0,8.0,4,47,01:52 PM,12:33 AM,06:22 AM,...,68,91,25.3,1014,25,8,169,7,San%20Francisco%20Bay%20Area,1
1,2008-07-10,25,16,0.0,8.0,4,47,01:52 PM,12:33 AM,06:22 AM,...,68,91,25.3,1014,25,8,169,7,San%20Francisco%20Bay%20Area,1
2,2014-06-30,30,22,0.0,10.0,6,16,08:49 AM,10:31 PM,05:34 AM,...,63,92,28.8,1006,30,7,198,16,Illinois,1
3,2014-07-01,26,18,0.0,15.0,6,23,09:46 AM,11:01 PM,05:34 AM,...,19,82,0.0,1007,26,10,265,17,Illinois,1
4,2014-07-02,18,14,0.0,5.0,4,30,10:42 AM,11:30 PM,05:35 AM,...,81,90,1.1,1015,18,7,297,13,Illinois,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
654,2009-03-08,24,19,0.0,12.0,6,81,04:08 PM,04:53 AM,06:37 AM,...,23,81,0.0,1017,24,10,164,16,Texas,0
655,2008-11-11,27,12,0.0,11.0,5,93,04:14 PM,04:22 AM,06:09 AM,...,22,79,0.0,1013,27,7,95,3,Alberta,0
656,2008-11-07,13,4,0.0,10.0,2,62,01:32 PM,No moonset,06:27 AM,...,33,65,0.0,1007,13,10,213,17,Michigan,0
657,2008-09-22,23,12,0.0,12.0,4,39,No moonrise,03:23 PM,07:03 AM,...,39,77,0.0,1027,23,7,58,9,Northern%20and%20Central%20California,0


In [55]:
concatenated_df['location'] = concatenated_df['location'].apply(urllib.parse.unquote)

In [56]:
concatenated_df

,date_time,maxtempC,mintempC,totalSnow_cm,sunHour,uvIndex,moon_illumination,moonrise,moonset,sunrise,...,cloudcover,humidity,precipMM,pressure,tempC,visibility,winddirDegree,windspeedKmph,location,target
0,2008-07-10,25,16,0.0,8.0,4,47,01:52 PM,12:33 AM,06:22 AM,...,68,91,25.3,1014,25,8,169,7,San Francisco Bay Area,1
1,2008-07-10,25,16,0.0,8.0,4,47,01:52 PM,12:33 AM,06:22 AM,...,68,91,25.3,1014,25,8,169,7,San Francisco Bay Area,1
2,2014-06-30,30,22,0.0,10.0,6,16,08:49 AM,10:31 PM,05:34 AM,...,63,92,28.8,1006,30,7,198,16,Illinois,1
3,2014-07-01,26,18,0.0,15.0,6,23,09:46 AM,11:01 PM,05:34 AM,...,19,82,0.0,1007,26,10,265,17,Illinois,1
4,2014-07-02,18,14,0.0,5.0,4,30,10:42 AM,11:30 PM,05:35 AM,...,81,90,1.1,1015,18,7,297,13,Illinois,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
654,2009-03-08,24,19,0.0,12.0,6,81,04:08 PM,04:53 AM,06:37 AM,...,23,81,0.0,1017,24,10,164,16,Texas,0
655,2008-11-11,27,12,0.0,11.0,5,93,04:14 PM,04:22 AM,06:09 AM,...,22,79,0.0,1013,27,7,95,3,Alberta,0
656,2008-11-07,13,4,0.0,10.0,2,62,01:32 PM,No moonset,06:27 AM,...,33,65,0.0,1007,13,10,213,17,Michigan,0
657,2008-09-22,23,12,0.0,12.0,4,39,No moonrise,03:23 PM,07:03 AM,...,39,77,0.0,1027,23,7,58,9,Northern and Central California,0


In [58]:
concatenated_df.to_csv('FinalDataSet.csv', index=False)